# COMP90051 Project 2
**Name:** *Kechen Zhao*

**Student ID:** *957398*

In [356]:
# You can add additional imports here
import numpy as np
from matplotlib import pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
import math


## 0. Loading the dataset

In [357]:
# do not edit this cell
# load the data files (download from the LMS)
embedded_images = np.load('images.npy')
labels = np.load('labels.npy')

# split into pool & testing
X_pool, X_test, y_pool, y_test = train_test_split(embedded_images, labels, 
                                                  test_size=0.5, random_state=1234, shuffle=True)

# sample a seed set
np.random.seed(1234)
label2id = defaultdict(list)
for i, label in enumerate(y_pool):
    label2id[label].append(i)
seed_set = []
for label, ids in label2id.items():
    seed_set.extend(np.random.choice(ids, size=10, replace=False))

In [383]:
# seed set
seed_X = []
seed_y = []
for index in seed_set:
    seed_X.append(X_pool[index])
    seed_y.append(y_pool[index])
# seed_X = pd.DataFrame(seed_X)
# seed_y = pd.DataFrame(seed_y).loc[:,0]

In [384]:
U_X = []
U_y = []
L_X = []
L_y = []
for index in range(len(X_pool)):
        if index not in seed_set:
            U_X.append(X_pool[index])
            U_y.append(y_pool[index])
        else:
            L_X.append(X_pool[index])
            L_y.append(y_pool[index])

In [387]:
import itertools
clustering = AgglomerativeClustering(n_clusters=30).fit(seed_X)
ii = itertools.count(len(seed_X))
tree = [{'node_id': next(ii), 'left': x[0], 'right':x[1]} for x in clustering.children_]
tree[-1]

{'node_id': 598, 'left': 596, 'right': 597}

In [380]:
# agglomerative clustering
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import AgglomerativeClustering
from matplotlib import pyplot

# define the model
model = AgglomerativeClustering(n_clusters=30)
# fit model and predict clusters
yhat = model.fit_predict(seed_X)
# retrieve unique clusters
clusters = unique(yhat)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = where(yhat == cluster)
    # create scatter of these samples
    #pyplot.scatter(X[row_ix, 0], X[row_ix, 1])
# show the plot
#pyplot.show()
model.distance_()

AttributeError: 'AgglomerativeClustering' object has no attribute 'distance_'

## 1. Applying logistic regression

In [361]:
#def train_logistic_regression(X, y, **args):
def train_logistic_regression(X, y, penalty, tol, C, max_iter, solver):
    """
    Train a logistic regression model on dataset (X, y) and return trained model.
    X: matrix of real values, size n x d
    y: vector of string labels, size n
    args: optional arguments e.g., for hyper-parameters
    """
    # your code here
    
    model = LogisticRegression(penalty='none',tol = tol, C = C, multi_class='multinomial', solver=solver, max_iter = max_iter)
    model.fit(X, y)
#     intercept = model.intercept_
#     coefficient = model.coef_.squeeze()
#     parameters = []
#     parameters.append(intercept)
#     parameters.append(coefficient)
    return model

    pass

In [362]:
def evaluate_logistic_regression_accuracy(Xt, yt, model):
    """
    Apply logistic regression prediction on dataset Xt and evaluate accuracy against yt,
    returing the accuracy results as a scalar.
    Xt: matrix of real values, size m x d
    yt: vector of string labels, size m
    """
    # your code here
    
    logmod = model
    #logmod.fit(Xt, yt)
    y_pred = logmod.predict(Xt)
    scores = metrics.accuracy_score(yt, y_pred)
    
    return scores

    pass

In [363]:
# hyper-parameter setup
penalty = 'none'
tol = 0.0001
C = 1.0

In [382]:
%%time
model = train_logistic_regression(seed_X, np.array(seed_y), penalty, tol, C, 100000, 'sag')
accuracy = evaluate_logistic_regression_accuracy(X_test, y_test, model)
print("Accuracy trained based on seed set:", accuracy)

Accuracy trained based on seed set: 0.312551867219917
CPU times: user 40.4 s, sys: 186 ms, total: 40.6 s
Wall time: 40.9 s


In [354]:
# modified version for query committee

def pool_based_active_learning(X_pool, y_pool, seed_ids,
                               train_func, select_func,
                               max_size, batch_size, select_method, query_method, **args):
    """
    Perform an active learning simulation, which starts by training on a seed set,
    then iteratively applies the selection function to rank instances in the pool,
    selects the top few instances which are included into the training set and the
    process repeats. 
        X_pool: matrix of n x d
        y_pool: vector of string labels, size n
        seed_ids: initial labelled set set, as a list of indices [0..n-1] into pool
        train_func: function which given (X, y, optional args) returns a trained model
        select_func: function which given (X, optional args) returns a sequence of scores
        max_size: stopping condition for active learning, when labelled data reaches given size
        batch_size: number of instances to be labelled in each iteration
        args: optional arguments passed to training and selection function
    returns the sequence of trained models 
    """
  
    # fill in
    
    # construct U, the pool of unlabelled instances and their hidden labels
    U_X = []
    U_y = []
    # construct L, the set of initial labelled instances
    L_X = []
    L_y = []
    
    for index in range(len(X_pool)):
        if index not in seed_ids:
            U_X.append(X_pool[index])
            U_y.append(y_pool[index])
        else:
            L_X.append(X_pool[index])
            L_y.append(y_pool[index])
#     L_X = pd.DataFrame(L_X)
#     L_y = pd.DataFrame(L_y).loc[:,0]

    
    b = batch_size
    T = int(max_size/batch_size)
    labels = set(y_pool)
    
    for t in range(0, T):
        # selection method
        if select_method == "query_by_committee":
            ensemble1, ensemble2, logmod = train_committee(np.array(L_X), np.array(L_y), 'none', 0.0001, 1.0, 20, 'sag')
            theta_t = logmod
            if query_method == "vote_entropy":
                model = [ensemble1, ensemble2]
                r = select_func(U_X, model, labels)
            else:
                model = [ensemble1, ensemble2]
                r = select_func(U_X, model)
        else:
            theta_t = train_func(np.array(L_X), np.array(L_y), 'none', 0.0001, 1.0, 20, 'sag')
            r = select_func(U_X, theta_t) # random_select
            
        # find the indices of top b instances in r
        max_index = []
        for i in range(b):
            max_index.append(r.index(max(r)))
            r[r.index(max(r))] = 0

        for j in max_index:
            x_j = U_X[j]
            L_X.append(np.array(x_j))
            U_X[j] = []
            y_j = U_y[j]
            L_y.append(np.array(y_j))
            U_y[j] = np.nan

        U_X = list(np.array(pd.DataFrame(U_X).dropna()))
        U_y = list(pd.DataFrame(U_y).dropna().loc[:,0])

    return theta_t
    pass

In [355]:
# vote_entropy
accuracy_vote_entropy = []
for size in size_list:
    max_size  = size
    models_us = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                     train_committee, 
                                     query_by_committee_vote_entropy, 
                                     max_size, batch, "query_by_committee", "vote_entropy")
    score = evaluate_logistic_regression_accuracy(X_test, y_test, models_us)
    accuracy_vote_entropy.append(score)
    print(f"Accuracy trained based on vote entropy selection with max_size {size}:", score)

9280
9220
Accuracy trained based on vote entropy selection with max_size 120: 0.3386929460580913
9280
9220
9160
9100
9040
Accuracy trained based on vote entropy selection with max_size 300: 0.3809128630705394
9280


KeyboardInterrupt: 

## 3. Uncertainty sampling

In [221]:
def logistic_regression_entropy_select(X, model, **args):
    """
    Given an unlabelled dataset X, a matrix of n x d, and a discriminative model 
    P(y|x), returns a vector of n entropy values.
    
    Should use intercept and coefficients from logistic regression to calculate the probability
    for each class y we have the corresponding intercept and coefficients
    
    X will be served as Data Frame
    
    """
    # fill in
    
    entropy = [];
    intercept = model.intercept_
    coefficient = model.coef_.squeeze()
    
    for instance in X:
        score = 0
        for i in range(len(intercept)):
            t = np.dot(coefficient[i], instance) + intercept[i]
            prob = 1/(1+np.exp(-t))
            score = score + prob*np.log(prob)
        entropy.append(-score)
        
    return entropy

    pass

In [223]:
## your code here for evaluation of accuracy and plotting of results

## 4. Query by committee

In [288]:
def query_by_committee_vote_entropy(X, model, labels, **args):
    
    """
    Given an unlabelled dataset X, a matrix of n x d, and a discriminative model 
    P(y|x), returns a vector of n vote entropy values.
    
    C = number of models
    
    Need to use C number of models to predict instances in X.
    Then for different classes of y, calculate the vote and sum them up.
    Assume argument model contains two models. 
    
    Arugment 'labels': a list of unique possible labels
    
    """
    
    X = pd.DataFrame(X)
    vote_entropy = []
    
    # get members in committee
    model1 = model[0]
    model2 = model[1]
    
    # use models to make predictions on the unlabelled dataset X
    classification1 = model1.predict(X)
    classification2 = model2.predict(X)
    

    # loop over each instances in X, calculate their corresponding vote entropy
    for i in range(len(X)):
        
        # create a dictionary to track the vote
        dic = dict.fromkeys(labels, 0)
        
        predic1 = classification1[i]
        predic2 = classification2[i]
        
        dic[predic1] = dic[predic1] + 1
        dic[predic2] = dic[predic2] + 1
        
        entropy = 0
        for vote in dic.values():
            if vote != 0:
                entropy = entropy + (vote/2)*np.log(vote/2)
            
        vote_entropy.append(-entropy)
        
    return vote_entropy

    pass


In [267]:
# # don't forget to provide function descriptive comments, like those provided in templates above

# def query_by_committee_vote_entropy(X, model, **args):
#     pass

# def query_by_committee_soft_vote_entropy(X, model, **args):
#     pass

# def query_by_committee_KL(X, model, **args):
#     pass

In [268]:
def train_committee(X, y, penalty, tol, C, max_iter, solver):
    
    """
    Train a logistic regression model on dataset (X, y) and return trained model.
    X: matrix of real values, size n x d
    y: vector of string labels, size n
    args: optional arguments e.g., for hyper-parameters
    C: number of models in committee
    ensemble_type: type of ensemble functions, e.g. LogisticRegressoin, SVM
    Could return two models:
    – the ensemble and the standard logistic regression model as a tuple 
    i.e., return (model1, model2)
    """

    ensemble1 = svm.SVC(probability=True, max_iter=200)
    ensemble1.fit(X,y)
    
    ensemble2 = LogisticRegression(multi_class='multinomial', max_iter = 20, solver='saga')
    ensemble2.fit(X,y)
        
    logmod = LogisticRegression(penalty='none',tol = tol, C = C, multi_class='multinomial', solver=solver, max_iter = max_iter)
    logmod.fit(X,y)
    
    return (ensemble1, ensemble2, logmod)

    pass

In [269]:
size_list = [120, 300, 600, 900, 1200, 1500, 1800, 2100, 2400, 2700, 3000]
batch = 60

In [345]:
# vote_entropy
accuracy_vote_entropy = []
for size in size_list:
    max_size  = size
    models_us = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                     train_committee, 
                                     query_by_committee_vote_entropy, 
                                     max_size, batch, "query_by_committee", "vote_entropy")
    score = evaluate_logistic_regression_accuracy(X_test, y_test, models_us)
    accuracy_vote_entropy.append(score)
    print(f"Accuracy trained based on vote entropy selection with max_size {size}:", score)

9280
9220
Accuracy trained based on vote entropy selection with max_size 120: 0.3411825726141079
9280
9220


TypeError: unhashable type: 'numpy.ndarray'

In [276]:
a = np.fromstring('0', dtype=int, sep=' ')
type(a)

numpy.ndarray

In [ ]:
## your code here for training, evaluation, and plotting code

## 5. Hierarchical sampling

In [ ]:
## your code